*package_leaflets_dataset_final.ipynb* from EZ --- creating json files in bioleaflets/dataset/test|train|valid   




## JSON FILE

            

    leaflet_content = {      
            'ID': leaflet.id,    
            'URL': leaflet.url,    
            'Product_Name': leaflet.product_name,    
            'Full_Content': leaflet.content,    

            'Section_1': {
                'Title': leaflet.section1.title,
                'Section_Content': leaflet.section1.section_content,
                'Entity_Recognition': leaflet.section1.entity_recognition
            },

            'Section_2': {
                'Title': leaflet.section2.title,
                'Section_Content': leaflet.section2.section_content,
                'Entity_Recognition': leaflet.section2.entity_recognition
            },

            'Section_3': {
                'Title': leaflet.section3.title,
                'Section_Content': leaflet.section3.section_content,
                'Entity_Recognition': leaflet.section3.entity_recognition
            },

            'Section_4': {
                'Title': leaflet.section4.title,
                'Section_Content': leaflet.section4.section_content,
                'Entity_Recognition': leaflet.section4.entity_recognition
            },

            'Section_5': {
                'Title': leaflet.section5.title,
                'Section_Content': leaflet.section5.section_content,
                'Entity_Recognition': leaflet.section5.entity_recognition
            },

            'Section_6': {
                'Title': leaflet.section6.title,
                'Section_Content': leaflet.section6.section_content,
                'Entity_Recognition': leaflet.section6.entity_recognition
            }
        }


In [1]:
import json
import os
import numpy as np

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

---------------------------------------------

### Read <test> T5_generations and Target Text

In [3]:
with open('human_evaluation_dataset/dataset_section1.json', 'r') as fp:
    dataset_section1 = json.load(fp)

In [4]:
# remove tags from entities

for id_index in dataset_section1:
    current_entities = dataset_section1[id_index]['input'].split()

    filtered_entities = []

    for entity in current_entities[2:]:
        if entity[0].isalpha() and entity[-1].isalpha():
            filtered_entities.append(entity)

    new_entities = " | ".join(filtered_entities)

    dataset_section1[id_index]['input'] = new_entities

In [5]:
dataset_section1["222"]['target']

"gefitinib mylan contains the active substance gefitinib which blocks a protein called ' epidermal growth factor receptor ' ( egfr ). this protein is involved in the growth and spread of cancer cells . gefitinib mylan is used to treat adults with non - small cell lung cancer . this cancer is a disease in which malignant ( cancer ) cells form in the tissues of the lung ."

In [8]:
# read generated sections
PATH_TEST_T5_GENERATED = 'results/T5_plain/outputs_seed_1/test_generations_beam_1.txt'

# read test dataset
with open(PATH_TEST_T5_GENERATED) as f:
    T5_test_generations = [line.strip() for line in f]

In [9]:
# read test dataset
# PATH_TEST_TARGET = 'scripts/data/plain/test.target'

# read T5 reference sections
# with open(PATH_TEST_TARGET) as f:
#     gold_reference_T5 = [line.strip() for line in f]
    
    
###### 
PATH_TEST_TARGET = 'results/test_truncated.target'

# read T5 reference sections
with open(PATH_TEST_TARGET) as f:
    gold_reference_T5 = [line.strip() for line in f]

In [10]:
# read test dataset
# PATH_TEST_SOURCE = 'scripts/data/plain/test.source'

# read T5 reference sections
# with open(PATH_TEST_SOURCE) as f:
#     T5_test_input = [line.strip() for line in f]


PATH_TEST_SOURCE = 'results/test_truncated.source'

with open(PATH_TEST_SOURCE) as f:
    T5_test_input = [line.strip() for line in f]

In [11]:
assert len(T5_test_generations) == len(gold_reference_T5)
assert len(T5_test_generations) == len(T5_test_input)

In [12]:
len(T5_test_generations)

742

#### TO-DO - make sure mapping is correct - it should be - but check

#### TO-DO - cut everthything to have length on similar scale - 512 tokens

-------------------------

## Building MVP

In [13]:
generations = T5_test_generations[:10]
targets = gold_reference_T5[:10]
inputs = T5_test_input[:10]

In [14]:
# generations with weights (default 100) - probability of being selected for evaluation

generations = {str(index):generations[index] for index in range(len(generations))}

targets = {str(index):targets[index] for index in range(len(targets))}

inputs = {str(index):inputs[index] for index in range(len(inputs))}

In [15]:
generations_weighted = {str(index):1/len(generations) for index in range(len(generations))}

### To-Do: Evaluate each generation just once! 

In [16]:
# save results here for faithfulness
# to what extend the generated text faithful to target output
# results_faithfulness = {key:list([]) for key in generations}

results_faithfulness = {str(index):list() for index in range(len(generations))}

In [17]:
results_faithfulness

{'0': [],
 '1': [],
 '2': [],
 '3': [],
 '4': [],
 '5': [],
 '6': [],
 '7': [],
 '8': [],
 '9': []}

In [18]:
list(generations_weighted.keys())

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [19]:
list(generations_weighted.values())

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

Making Probabilities work

In [16]:
0.1 / 2

0.05

In [17]:
sum(list(generations_weighted.values()))

0.9999999999999999

In [18]:
generated_sample_index = np.random.choice(list(generations_weighted.keys()), 
                                              size=1,
                                              replace=True,
                                              p=list(generations_weighted.values()))

In [19]:
generated_sample_index

array(['2'], dtype='<U1')

In [20]:
generated_sample_index[0]

'2'

In [21]:
generations_weighted["2"] = generations_weighted["2"] / 2

In [22]:
list(generations_weighted.values())

[0.1, 0.1, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [23]:
sum(list(generations_weighted.values()))

0.9499999999999998

In [24]:
(1 - sum(list(generations_weighted.values()))) / (len(generations)-1)

0.005555555555555573

In [25]:
num = (1 - sum(list(generations_weighted.values()))) / (len(generations)-1)

In [26]:
num

0.005555555555555573

In [27]:
# [generations_weighted[index] += num if index =! '2' for index in list(generations_weighted.keys())]

In [28]:
for index in list(generations_weighted.keys()):
    if index != '2':
        generations_weighted[index] += num

In [29]:
list(generations_weighted.values())

[0.10555555555555558,
 0.10555555555555558,
 0.05,
 0.10555555555555558,
 0.10555555555555558,
 0.10555555555555558,
 0.10555555555555558,
 0.10555555555555558,
 0.10555555555555558,
 0.10555555555555558]

In [30]:
sum(list(generations_weighted.values()))

1.0000000000000004

## Human Evaluation Part

At the most basic level, interact autogenerates UI controls for function arguments, and then calls the function with those arguments when you manipulate the controls interactively. 

In [15]:
# add slide bar to the function with the interact decorator.    
@interact(faithfulness=(0, 10))
def human_evaluation(faithfulness):
    
    
    
    # randomly select a sample according to weighted distribution
    generated_sample_index = np.random.choice(list(generations_weighted.keys()), 
                                              size=1,
                                              replace=True,
                                              p=list(generations_weighted.values()))
    
    # '3' not array(['3']
    generated_sample_index = generated_sample_index[0]
    
    # update the weight of selected sample (decrease by half)
    generations_weighted[generated_sample_index] = generations_weighted[generated_sample_index] / 2
    
    # evenly increase all other probabilities
    increase_num = (1 - sum(list(generations_weighted.values()))) / (len(generations)-1)
    
    for another_index in list(generations_weighted.keys()):
        if another_index != generated_sample_index:
            generations_weighted[another_index] += increase_num
    
    # display input-target-generation
    print("~~~~~~~~ Input ~~~~~~~")
    print(inputs[generated_sample_index])
    
    print("\n~~~~~~~~ Generation ~~~~~~~")
    print(generations[generated_sample_index])
    
    print("\n~~~~~~~~ Target ~~~~~~~")
    print(targets[generated_sample_index])
    
    
    # used selects a scale
    
    # save results
    results_faithfulness[generated_sample_index].append(faithfulness)
    
    # print("\n\nThe square of %d is %d." % (faithfulness, faithfulness**2))

interactive(children=(IntSlider(value=5, description='faithfulness', max=10), Output()), _dom_classes=('widget…

In [30]:
generations_weighted

{'0': 0.06571812117745549,
 '1': 0.11112268226647644,
 '2': 0.08328776225386743,
 '3': 0.06465420433741724,
 '4': 0.11361957655168947,
 '5': 0.17499561403661706,
 '6': 0.06399825016935448,
 '7': 0.1779884231340167,
 '8': 0.07188703653965377,
 '9': 0.07272832953345185}

In [31]:
results_faithfulness

{'0': [9, 1, 4, 6, 5],
 '1': [9, 5],
 '2': [7, 6, 8, 1],
 '3': [9, 8, 8, 7, 6],
 '4': [1, 5],
 '5': [4, 7],
 '6': [5, 4, 5, 8],
 '7': [5, 9],
 '8': [4, 1, 9],
 '9': [1, 6, 2, 3]}

In [21]:
# TO-DO- truncate text
# use code from presentation_demo to make text - "untokenized"
# think how to save results
# maybe not a number but a button from 1-10 would be easier to choose!!!

# have problem in prob dist. I need not to update evenly all samples excpet the current, 
# but update *5 less probable* 


# instead of T5_Tokenizer threshold, choose another threshold ~~~~ Similar size in length (char-wise, token-wise)
# hughlighting NER

## Question - Is this text adequate compared to original text? (1-10) ~ Everything

Fluency?   
Hallucination present?     
Missing Fact?     
Typos?     
Repetitions?    

#### How sever are hallucinations?

#### To which extend all entities included in the generation? 
(How sever are missing facts? Are all entities included in the generation?)

#### How fluent is this generation?

Just 1 section - specialize in section 1 writing :)  & intermediate results


T5/BART/Content Planner - mixed

https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html   
https://aclanthology.org/2020.coling-main.420.pdf   


https://www.dev2qa.com/how-to-add-interactive-widget-slide-bar-in-jupyter-notebook/    

jupyter notebook ipywidgets sliding bar    

https://ipywidgets.readthedocs.io/en/stable/    
https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html    

--------------------------------------------

In [3]:
from IPython.display import display

In [27]:
one_var_array = []
two_var_array = []

In [29]:

button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

In [30]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

In [37]:
w = widgets.Dropdown(
    options=['Addition', 'Multiplication', 'Subtraction', 'Division', "Suck_my_dick"],
    value='Addition',
    description='Task:',
)

def on_change(change):
    
#     print(change)
#     print()
    
    print(change['type'])
    print(change['new'])
    
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

w.observe(on_change)

display(w)

Dropdown(description='Task:', options=('Addition', 'Multiplication', 'Subtraction', 'Division', 'Suck_my_dick'…

change
{'index': 4}
change
Suck_my_dick
change
Suck_my_dick
changed to Suck_my_dick
change
4
change
{}


In [98]:
adequacy_resuls = []
faithfulness_results = []

In [99]:
style = {'description_width': 'initial'}

In [111]:
button_custom = widgets.Button(description="Suka")

def on_button_clicked_suka(b):
    print("Button Suka clicked.")

button_custom.on_click(on_button_clicked_suka)

In [112]:
display(button_custom)


Button(description='Suka', style=ButtonStyle())

Button Suka clicked.
Button Suka clicked.
Button Suka clicked.
Button Suka clicked.
Button Suka clicked.
Button Suka clicked.
Button Suka clicked.
Button Suka clicked.


## --------------------------------------------------------------------------------

In [6]:
style = {'description_width': 'initial'}

In [46]:
adequacy_resuls = []
hallucination_results = []
all_entities_results = []
fluency_results = []

In [55]:
main_button = widgets.Button(description="Show me a Sample")

dropdown_general = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='To which extent is this text *adequate*? ',
                style=style,
)

dropdown_hallucination = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='How severe are *hallucinations*? ',
                style=style,
)

dropdown_all_entities = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='To which extend *all entities* included?: ',
                style=style,
)


dropdown_fluency = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='How *fluent* is the generation?: ',
                style=style,
)

In [49]:
def on_change_dropdown_general(change):
    if change['type'] == 'change' and change['name'] == 'value':
        adequacy_resuls.append(change['new'])

        
def on_change_dropdown_hallucination(change):
    if change['type'] == 'change' and change['name'] == 'value':
        hallucination_results.append(change['new'])
        
        
def on_change_dropdown_all_entities(change):
    if change['type'] == 'change' and change['name'] == 'value':
        all_entities_results.append(change['new'])
        

def on_change_dropdown_fluency(change):
    if change['type'] == 'change' and change['name'] == 'value':
        fluency_results.append(change['new'])

In [50]:
dropdown_general.observe(on_change_dropdown_general)
dropdown_hallucination.observe(on_change_dropdown_hallucination)
dropdown_all_entities.observe(on_change_dropdown_all_entities)
dropdown_fluency.observe(on_change_dropdown_fluency)

In [51]:
def on_button_clicked(b):
    
    # check that user selected all dropdowns
    assert len(adequacy_resuls) == len(hallucination_results)
    assert len(adequacy_resuls) == len(all_entities_results)
    assert len(adequacy_resuls) == len(fluency_results)
    
    # display text
    update_text()
    
    # display dropdowns
    display(dropdown_general)
    display(dropdown_hallucination)
    display(dropdown_all_entities)
    display(dropdown_fluency)
    
    # clear output
    # output_screen.clear_output()
    # IPython.display.clear_output
    
    # display again button
    display(main_button)

    
def update_text():
    print('Random Num 1: ', np.random.randint(1,100))
    print('Random Num 2: ', np.random.randint(100,10000))

In [52]:
main_button.on_click(on_button_clicked)

In [53]:
display(main_button)

Button(description='Show me a Sample', style=ButtonStyle())

Random Num 1:  97
Random Num 2:  5718


Dropdown(description='To which extent is this text *adequate*? ', options=('Not selected', '1 One 1', '2 Two 2…

Dropdown(description='How severe are *hallucinations*? ', options=('Not selected', '1 One 1', '2 Two 2', '3 Th…

Dropdown(description='To which extend *all entities* included?: ', options=('Not selected', '1 One 1', '2 Two …

...

Dropdown(description='How *fluent* is the generation?: ', index=7, options=('Not selected', '1 One 1', '2 Two …

Button(description='Show me a Sample', style=ButtonStyle())

AssertionError: 

AssertionError: 

AssertionError: 

In [54]:
print(adequacy_resuls)
print(hallucination_results)
print(all_entities_results)
print(fluency_results)

['2 Two 2', '10 Ten 10', '3 Three 3']
['7 Seven 7', '9 Nine 9', '3 Three 3']
['6 Six 6', '8 Eight 8']
['7 Seven 7']


In [123]:
user_input = input("\nSee a new sample? (just press <Enter>)\n")


See a new sample? (just press <Enter>)



In [137]:
user_input = input("\nSee a new sample? (just press <Enter>)\n")

while user_input not in ['n', 'no', 'please no', 'nope', 'nein', 'leave me alone']:
    display(main_button)
    
    user_input = input("\nSee a new sample?\n")


See a new sample? (just press <Enter>)



Button(description='Update Sample', style=ButtonStyle())


See a new sample?
no


In [128]:
display(main_button)

Button(description='Update Sample', style=ButtonStyle())

## --------------------------------------------------------------------------------

In [100]:
#############################################

button = widgets.Button(description="Update Sample")

dropdown_general = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='To which extent is this text *adequate*? ',
                style=style,
)

dropdown_faithfulness = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='How severe are *hallucinations*? ',
                style=style,
)


def on_button_clicked(b):
    
    update_text()
    
    # TO-DO - RULE - make sure all dropdowns selected!!!
    
    print("Button clicked.")
    

def update_text():
    print('Random Num 1: ', np.random.randint(1,100))
    print('Random Num 2: ', np.random.randint(100,10000))
    

def on_change_dropdown_general(change):
    
    if change['type'] == 'change' and change['name'] == 'value':
        # print("changed to %s" % change['new'])
        adequacy_resuls.append(change['new'])


def on_change_dropdown_faithfulness(change):
    
    if change['type'] == 'change' and change['name'] == 'value':
        faithfulness_results.append(change['new'])

        
        
dropdown_general.observe(on_change_dropdown_general)
dropdown_faithfulness.observe(on_change_dropdown_faithfulness)


print('Random Num 1: ', np.random.randint(1,100))
print('Random Num 2: ', np.random.randint(100,10000))


display(dropdown_general)
display(dropdown_faithfulness)




display(button)


button.on_click(on_button_clicked)

##############################################

Random Num 1:  74
Random Num 2:  6016


Dropdown(description='To which extent is this text *adequate*? ', options=('Not selected', '1 One 1', '2 Two 2…

Dropdown(description='How severe are *hallucinations*? ', options=('Not selected', '1 One 1', '2 Two 2', '3 Th…

Button(description='Update Sample', style=ButtonStyle())

Random Num 1:  67
Random Num 2:  6841
Button clicked.
Random Num 1:  94
Random Num 2:  6258
Button clicked.


In [104]:
print(adequacy_resuls)
print(faithfulness_results)

['1 One 1']
['10 Ten 10']


In [ ]:
#### How sever are hallucinations?

#### To which extend all entities included in the generation? 
(How sever are missing facts? Are all entities included in the generation?)

#### How fluent is this generation?

In [ ]:
#### Button try


In [24]:
@interact(faithfulness=[('1 One 1', 1), ('2 Two 2', 2), ('3 Three 3', 3), ('4 Four 4', 4), ('5 Five 5', 5), 
                        ('6 Six 6', 6), ('7 Seven 7', 7), ('8 Eight 8', 8), ('9 Nine 9', 20), ('10 Ten 10', 10)],
         entities_score=[('1 One 1', 1), ('2 Two 2', 2), ('3 Three 3', 3), ('4 Four 4', 4), ('5 Five 5', 5)])
def eval_faith(faithfulness, entities_score):
    
    print('How sever are hallucinations?')
    print('\t\t To which extend all entities included in the generation?')
    
    one_var = np.random.randint(1,100)
    print(one_var)
    
    two_var = np.random.randint(100,1000)
    print(two_var)
    
    # save results
    one_var_array.append(faithfulness)
    two_var_array.append(entities_score)

interactive(children=(Dropdown(description='faithfulness', options=(('1 One 1', 1), ('2 Two 2', 2), ('3 Three …

In [25]:
one_var_array

[1, 5, 8, 2, 2, 2, 1, 4, 5]

In [26]:
two_var_array

[1, 1, 1, 1, 5, 1, 1, 1, 1]

In [23]:
widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
#     value='pineapple',
    description='Pizza topping:',
    disabled=False
)

RadioButtons(description='Pizza topping:', options=('pepperoni', 'pineapple', 'anchovies'), value='pepperoni')

In [ ]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

In [18]:
# add another slide bar to this function with the interact decorator.    
@interact(y=(0, 10))
def square_number(y):
    print("The square of %d is %d." % (y, y**2))

interactive(children=(IntSlider(value=5, description='y', max=10), Output()), _dom_classes=('widget-interact',…

--------------------

In [5]:
def f(x):
    print(x)
    return x

In [6]:
interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

### Meeting Angelo Tuesday

In [ ]:
# add slide bar to the function with the interact decorator.    
@interact(faithfulness=(0, 10))
def human_evaluation(faithfulness):
    
    # randomly select a sample according to weighted distribution
    generated_sample_index = np.random.choice(list(generations_weighted.keys()), 
                                              size=1,
                                              replace=True,
                                              p=list(generations_weighted.values()))
    
    # '3' not array(['3']
    generated_sample_index = generated_sample_index[0]
    
    # update the weight of selected sample (decrease by half)
    generations_weighted[generated_sample_index] = generations_weighted[generated_sample_index] / 2
    
    # evenly increase all other probabilities
    increase_num = (1 - sum(list(generations_weighted.values()))) / (len(generations)-1)
    
    for another_index in list(generations_weighted.keys()):
        if another_index != generated_sample_index:
            generations_weighted[another_index] += increase_num
    
    # display input-target-generation
    print("~~~~~~~~ Input ~~~~~~~")
    print(inputs[generated_sample_index])
    
    print("\n~~~~~~~~ Generation ~~~~~~~")
    print(generations[generated_sample_index])
    
    print("\n~~~~~~~~ Target ~~~~~~~")
    print(targets[generated_sample_index])
    
    
    # used selects a scale
    
    # save results
    results_faithfulness[generated_sample_index].append(faithfulness)
    
    # print("\n\nThe square of %d is %d." % (faithfulness, faithfulness**2))

In [ ]:
generations_weighted

In [ ]:
results_faithfulness

In [ ]:
### Keep Rolling

#############################################

button = widgets.Button(description="Update Sample")

dropdown_general = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='To which extent is this text *adequate*? ',
                style=style,
)

dropdown_faithfulness = widgets.Dropdown(
                options=['Not selected', '1 One 1', '2 Two 2', '3 Three 3', '4 Four 4', '5 Five 5', 
                         '6 Six 6', '7 Seven 7', '8 Eight 8', '9 Nine 9', '10 Ten 10'],
                value='Not selected',
                description='How severe are *hallucinations*? ',
                style=style,
)


def on_button_clicked(b):
    
    update_text()
    
    # TO-DO - RULE - make sure all dropdowns selected!!!
    
    print("Button clicked.")
    

def update_text():
    print('Random Num 1: ', np.random.randint(1,100))
    print('Random Num 2: ', np.random.randint(100,10000))
    

def on_change_dropdown_general(change):
    
    if change['type'] == 'change' and change['name'] == 'value':
        # print("changed to %s" % change['new'])
        adequacy_resuls.append(change['new'])


def on_change_dropdown_faithfulness(change):
    
    if change['type'] == 'change' and change['name'] == 'value':
        faithfulness_results.append(change['new'])

        
        
dropdown_general.observe(on_change_dropdown_general)
dropdown_faithfulness.observe(on_change_dropdown_faithfulness)


print('Random Num 1: ', np.random.randint(1,100))
print('Random Num 2: ', np.random.randint(100,10000))


display(dropdown_general)
display(dropdown_faithfulness)




display(button)


button.on_click(on_button_clicked)

##############################################

Google Colab Work

In [ ]:
def get_random_sample_id(weights_samples):

  section_ids = list(weights_samples.keys())
  weights_ids = [weights_samples[id] for id in section_ids]

  # no samples to choose - GAME over
  if sum(weights_ids) == 0: 
    print("\n\tYou are the BEST, You DID it! Congratulations and many thanks!")
    return None

  # prob must sum up to 1
  assert round(sum(weights_ids), 10) == 1, "Probs do NOT sum up to 1"

  # randomly pick up a sample id
  generated_sample_index = np.random.choice(section_ids, 
                                            size=1,
                                            replace=True,
                                            p=weights_ids)

  # '3' not array(['3']
  generated_sample_index = generated_sample_index[0]

  # set prob to 0 of the selected section_id
  weights_samples[generated_sample_index] = 0

  # update other probs (not zero!) so that sum(prob) = 1
  # do not update those prob that are already 0

  # count not 0 values - we do not want to update those with already 0
  non_zero_ids = len([id for id in weights_samples if weights_samples[id] != 0]) 

  # if the last element non_zero_ids = 0
  if non_zero_ids != 0: increase_num = (1 - sum(list(weights_samples.values()))) / non_zero_ids
  else: increase_num = 0

  # update other probs
  for sample_id in list(weights_samples.keys()):
      # do not update already ids with 0 and current id
      if sample_id != generated_sample_index and weights_samples[sample_id] != 0:
          weights_samples[sample_id] += increase_num

  # print(generated_sample_index, " = ", non_zero_ids)
  return generated_sample_index

In [ ]:
#### Check the sampling function

check_ids = []

for i in range(len(weights_samples)):

  ans = get_random_sample_id(weights_samples)
  check_ids.append(ans)

# check
assert len(check_ids) == len(weights_samples)

In [ ]:
# from start - weight for each sample are uniformly distributed
# weights_samples = {str(index):1/len(dataset_section1) for index in range(1,len(dataset_section1)+1)}

#### Build with :

https://github.com/jupyter-widgets/ipywidgets/tree/e0d41f6f02324596a282bc9e4650fd7ba63c0004/docs/source/examples   
https://github.com/jupyter-widgets/ipywidgets/issues/2103      
https://stackoverflow.com/questions/34020789/ipywidgets-dropdown-widgets-what-is-the-onchange-event    
https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html    
https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#SelectMultiple     
https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html     

